In [ ]:
# Standard library imports
import math
import pickle
import random
import time

# Third-party library imports
import matplotlib as mpl
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from scipy.interpolate import griddata
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from skimage.metrics import structural_similarity as ssim

In [ ]:
def x_to_y(X): # averaging in 2*2 windows (4 pixels)
    dim = X.shape[0]
    dim = 20
    Y = np.zeros((int(dim/2),int(dim/2)))
    for i in range(int(dim/2)):
        for j in range(int(dim/2)):
            Y[i,j] = X[2*i,2*j] + X[2*i+1,2*j] + X[2*i,2*j+1] + X[2*i+1,2*j+1]

            Y_noise = np.random.multivariate_normal(np.zeros(100),0.0000 * np.eye(100))
            Y_noise.shape = (10,10)
            Y = Y + Y_noise
    return Y


class shallow(object):

    time = 0

    plt = []
    fig = []


    def __init__(self, x=[],y=[],h_ini = 1.,u=[],v = [],dx=0.01,dt=0.0001, N=64,L=1., px=16, py=16, R=64, Hp=0.1, g=1., b=0.): # How define no default argument before?


        # add a perturbation in pressure surface


        self.px, self.py = px, py
        self.R = R
        self.Hp = Hp



        # Physical parameters

        self.g = g
        self.b = b
        self.L=L
        self.N=N

        self.dx=dx
        self.dt=dt

        self.x,self.y = np.mgrid[:self.N,:self.N]

        self.u=np.zeros((self.N,self.N))
        self.v=np.zeros((self.N,self.N))

        self.h_ini=h_ini

        self.h=self.h_ini * np.ones((self.N,self.N))

        rr = (self.x-px)**2 + (self.y-py)**2
        self.h[rr<R] = self.h_ini + Hp #set initial conditions

        self.lims = [(self.h_ini-self.Hp,self.h_ini+self.Hp),(-0.02,0.02),(-0.02,0.02)]



    def dxy(self, A, axis=0):
        """
        Compute derivative of array A using balanced finite differences
        Axis specifies direction of spatial derivative (d/dx or d/dy)
        dA[i]/dx =  (A[i+1] - A[i-1] )  / 2dx
        """
        return (np.roll(A, -1, axis) - np.roll(A, 1, axis)) / (self.dx*2.) # roll: shift the array axis=0 shift the horizontal axis

    def d_dx(self, A):
        return self.dxy(A,1)

    def d_dy(self, A):
        return self.dxy(A,0)


    def d_dt(self, h, u, v):
        """
        http://en.wikipedia.org/wiki/Shallow_water_equations#Non-conservative_form
        """
        for x in [h, u, v]: # type check
           assert isinstance(x, np.ndarray) and not isinstance(x, np.matrix)

        g,b,dx = self.g, self.b, self.dx

        du_dt = -g*self.d_dx(h) - b*u
        dv_dt = -g*self.d_dy(h) - b*v

        H = 0 #h.mean() - our definition of h includes this term
        dh_dt = -self.d_dx(u * (H+h)) - self.d_dy(v * (H+h))

        return dh_dt, du_dt, dv_dt


    def evolve(self):
        """
        Evolve state (h, u, v) forward in time using simple Euler method
        x_{N+1} = x_{N} +   dx/dt * d_t
        """

        dh_dt, du_dt, dv_dt = self.d_dt(self.h, self.u, self.v)
        dt = self.dt

        self.h += dh_dt * dt
        self.u += du_dt * dt
        self.v += dv_dt * dt
        self.time += dt

        return self.h, self.u, self.v

In [ ]:
import random as rand
def simu(iteration_times, Hp, R, n_steps,blank_steps, px, py):
    SW = shallow(N=64, px=px, py=py, R=R, Hp=Hp, b=0.2)
    num = (iteration_times - blank_steps) // n_steps
    true_u_vect = np.zeros((num, SW.N, SW.N))
    true_v_vect = np.zeros((num, SW.N, SW.N))
    true_h_vect = np.zeros((num, SW.N, SW.N))
    index = 0

    for i in range(iteration_times):
        SW.evolve()

        if i % n_steps == 0 and i >= 500:
            true_u_vect[index], true_v_vect[index], true_h_vect[index] = SW.u, SW.v, SW.h
            index += 1

    return true_u_vect, true_v_vect, true_h_vect


def structure_obs(N, random_range):
    x = np.arange(0, N, 7)
    y = np.arange(0, N, 7)
    x_list, y_list = np.meshgrid(x, y)

    x_coord = np.empty(x_list.size, dtype=int)
    y_coord = np.empty(y_list.size, dtype=int)

    for i, (x_val, y_val) in enumerate(zip(x_list.flatten(), y_list.flatten())):
        if (i+1) % 64 != 0:
            x_val += rand.randint(0, random_range)
            y_val += rand.randint(0, random_range)

        x_coord[i] = x_val if x_val < N else N - 1
        y_coord[i] = y_val if y_val < N else N - 1

    return x_coord, y_coord

In [ ]:
random.seed(112)
second_element_range = (0.2, 0.8)
third_element_range = (4, 12)

# Number of random tuples to generate
num_random_tuples = 50
steps = 3500
blank_steps = 500
px = 32
py = 32
# Generate random tuples
sim_params = [(steps,
               round(random.uniform(*second_element_range), 2),
               random.randint(*third_element_range),
               10,
               blank_steps,
               px,#np.random.randint(30, 34),  # px
               py)#random.randint(30, 34))  # py
              for _ in range(num_random_tuples)]
# Simulate data and stack the results
def simulate_and_stack(sim_params):
    true_u_vect, true_v_vect, true_h_vect = [], [], []
    for nsteps, hp, r, n,blank,px,py in sim_params:
        u, v, h = simu(nsteps, hp, r**2, n,blank,px,py)
        true_u_vect.append(u)
        true_v_vect.append(v)
        true_h_vect.append(h)

    true_u_vect = np.vstack(true_u_vect)
    true_v_vect = np.vstack(true_v_vect)
    true_h_vect = np.vstack(true_h_vect)

    return true_u_vect, true_v_vect, true_h_vect

# Simulate data and stack the results
true_u_vect, true_v_vect, true_h_vect = simulate_and_stack(sim_params)

In [ ]:

# Extract data for train_params
second_elements_train = [param[1] for param in train_params]
third_elements_train = [param[2] for param in train_params]
df_train = pd.DataFrame(list(zip(second_elements_train, third_elements_train)), columns=['Second Element', 'Third Element'])
frequency_train = df_train.groupby(['Second Element', 'Third Element']).size().reset_index(name='Frequency')
x_train = frequency_train['Second Element']
y_train = frequency_train['Third Element']

# Extract data for test_params
second_elements_test = [param[1] for param in test_params]
third_elements_test = [param[2] for param in test_params]
df_test = pd.DataFrame(list(zip(second_elements_test, third_elements_test)), columns=['Second Element', 'Third Element'])
frequency_test = df_test.groupby(['Second Element', 'Third Element']).size().reset_index(name='Frequency')
x_test = frequency_test['Second Element']
y_test = frequency_test['Third Element']

# Set up the figure and axes for side-by-side plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot for train_params
scatter1 = ax1.scatter(x_train, y_train, c='turquoise', edgecolor='black', s=150, alpha=0.75)
ax1.set_xlabel('Water Height (m)', fontsize=12)
ax1.set_ylabel('Radius (grid unit)', fontsize=12)
ax1.set_title('Training Parameter Combinations', fontsize=14)
ax1.grid(True, linestyle='--')
ax1.set_facecolor('whitesmoke')

# Plot for test_params
scatter2 = ax2.scatter(x_test, y_test, c='purple', edgecolor='black', s=150, alpha=0.75)
ax2.set_xlabel('Water Height (m)', fontsize=12)
ax2.set_ylabel('Radius (grid unit)', fontsize=12)
ax2.set_title('Testing Parameter Combinations', fontsize=14)
ax2.grid(True, linestyle='--')
ax2.set_facecolor('whitesmoke')

plt.tight_layout()
plt.show()

In [ ]:
np.save('./Dataset_sw/true_u_full.npy',true_u_vect)
np.save('./Dataset_sw/true_v_full.npy',true_v_vect)
np.save('./Dataset_sw/true_h_full.npy',true_h_vect)

In [ ]:
true_u_vect = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/true_u_full.npy', mmap_mode="r")
true_v_vect = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/true_v_full.npy', mmap_mode="r")
true_h_vect = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/true_h_full.npy', mmap_mode="r")

a = 50*300


# Constants
N = 64
datasize = int(a)  # Ensure 'a' is defined earlier in your code

# Initialize arrays
true_data = np.zeros((datasize, N, N, 3))  # Combining u, v, h data into one array for simplicity
vor_data = np.zeros((datasize, N, N, 3))

# Grid preparation
x, y = np.linspace(0., N, N), np.linspace(0., N, N)[::-1]
X, Y = np.meshgrid(x, y)
P = np.column_stack((X.flatten(), Y.flatten()))

# Data processing and conditional plotting
for index in range(datasize):
    for vect, slice_idx in zip([true_u_vect, true_v_vect, true_h_vect], range(3)):
        x_coord, y_coord = structure_obs(64, 2)
        Zi = vect[index, x_coord.astype(int), N-1-y_coord.astype(int)]
        Pi = np.column_stack((x_coord, y_coord))
        Z_nearest = griddata(Pi, Zi, P, method="nearest").reshape([N, N])
        vor_data[index, :, :, slice_idx] = Z_nearest
        true_data[index, :, :, slice_idx] = vect[index, :, :]

# Correcting the data swap issue
vor_u_data, vor_v_data = vor_data[..., 0], vor_data[..., 1]
vor_h_data, true_u_data = vor_data[..., 2], true_data[..., 0]
true_v_data, true_h_data = true_data[..., 1], true_data[..., 2]

np.save('./Dataset_sw/vor_u_full.npy', vor_u_data)
np.save('./Dataset_sw/vor_v_full.npy', vor_v_data)
np.save('.Dataset_sw/vor_h_full.npy', vor_h_data)


In [ ]:
vor_u_data = np.load('./Dataset_sw/vor_u_full.npy', mmap_mode="r")
vor_v_data = np.load('./Dataset_sw/vor_v_full.npy', mmap_mode="r")
vor_h_data = np.load('./Dataset_sw/vor_h_full.npy', mmap_mode="r")
true_v_data = np.load('./Dataset_sw/true_v_full.npy', mmap_mode="r")
true_u_data = np.load('./Dataset_sw/true_u_full.npy', mmap_mode="r")
true_h_data = np.load('./Dataset_sw/true_h_full.npy', mmap_mode="r")

In [ ]:
datasize = 50 * 300
train_rate = 0.80

# Directly calculate train and test sizes
train_size = int(datasize * train_rate)
test_size = datasize - train_size

# Data categories and datasets
data_types = ['u', 'v', 'h']
voronoi_data = [vor_u_data, vor_v_data, vor_h_data]
true_data = [true_v_data, true_u_data, true_h_data] 

def split_data(data, train_size):
    return data[:train_size], data[train_size:]

vor_train, vor_test = zip(*[split_data(data, train_size) for data in voronoi_data])
true_train, true_test = zip(*[split_data(data, train_size) for data in true_data])

vor_train = np.stack(vor_train, axis=-1)
true_train = np.stack(true_train, axis=-1)
vor_test = np.stack(vor_test, axis=-1)
true_test = np.stack(true_test, axis=-1)

In [ ]:
save_path = './Dataset_sw/'

# Save the arrays to disk
np.save(save_path + 'vor_train.npy', vor_train)
np.save(save_path + 'true_train.npy', true_train)
np.save(save_path + 'vor_test.npy', vor_test)
np.save(save_path + 'true_test.npy', true_test)